In [2]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
input_file = 'clean_data.csv'
file = pandas.read_csv(input_file)

In [4]:
import gensim
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

Using TensorFlow backend.


In [5]:
#为每条tweet加上标签并分词,label为标签前缀
def labelize_split_tweets(tweets, label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_' + str(i)]))
    return result

In [6]:
x = file.Tweet
# 对x进行shuffle
x = x.sample(frac=1)
x_split = labelize_split_tweets(x, 'all')
x.head()

612083     Had to go to the resource fair last night... l...
1399621    i think.....im gonna go chilli at the lake...w...
414062     calling me that was so not the right thing to ...
1565410    Superman!!!!!  Hmmm, is it Super heroes that u...
546604                                    feels down again  
Name: Tweet, dtype: object

In [40]:
tqdm.pandas(desc="progress-bar")
core_num = multiprocessing.cpu_count()

下面对数据集进行word2vec，利用cbow和skip-gram两种模型进行生成。此处所用到的参数值参考了参考资料。后面还会探究词向量长度对训练结果的影响。

In [ ]:
%%time
model_cbow_100 = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2, workers=core_num, alpha=0.065)
model_cbow_100.build_vocab([x.words for x in tqdm(x_split)])
for epoch in range(30):
    model_cbow_100.train(utils.shuffle([x.words for x in tqdm(x_split)]), total_examples=len(x_split), epochs=1)
    model_cbow_100.alpha -= 0.002
    model_cbow_100.min_alpha =model_cbow.alpha
model_cbow.save('model_cbow_100.word2vec')

100%|██████████| 1597125/1597125 [00:00<00:00, 2142188.08it/s]


In [ ]:
%%time
model_sg_100 = Word2Vec(sg=1, size=100, negative=5, window=2, min_count=2, workers=core_num, alpha=0.065)
model_sg_100.build_vocab([x.words for x in tqdm(x_split)])
for epoch in range(30):
    model_sg_100.train(utils.shuffle([x.words for x in tqdm(x_split)]), total_examples=len(x_split), epochs=1)
    model_sg_100.alpha -= 0.002
    model_sg_100.min_alpha =model_cbow.alpha
model_cbow.save('model_sg_100.word2vec')